In [1]:
import numpy as np
import pandas as pd
from keras.applications.vgg16 import VGG16, preprocess_input

In [2]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [3]:
for i in range(len(vgg16.layers)):
    if i>16:
        vgg16.layers[i].trainable = True
    else:
        vgg16.layers[i].trainable = False

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
Total params: 40,930,624
Trainable params: 28,575,744
Non-trainable params: 12,354,880
_________________________________________________________________


In [6]:
from keras.optimizers import Adam

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'train2014',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


# Evaluate the model
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
        'val2014',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')



Found 82783 images belonging to 1 classes.
Found 40504 images belonging to 1 classes.


In [8]:
# Fine-tune the model
model.fit(
        train_generator,
        steps_per_epoch=50,
        epochs=10,
        validation_data=val_generator,
        validation_steps=20)

Epoch 1/10
50/50 [==============================] - 906s 18s/step - loss: 3849.6396 - accuracy: 0.0000e+00 - val_loss: 3677.6860 - val_accuracy: 0.0000e+00
Epoch 2/10
50/50 [==============================] - 849s 17s/step - loss: 3647.3516 - accuracy: 0.0000e+00 - val_loss: 3632.1421 - val_accuracy: 0.0000e+00
Epoch 3/10
50/50 [==============================] - 852s 17s/step - loss: 3595.9253 - accuracy: 0.0000e+00 - val_loss: 3561.3547 - val_accuracy: 0.0000e+00
Epoch 4/10
50/50 [==============================] - 849s 17s/step - loss: 3538.0107 - accuracy: 0.0000e+00 - val_loss: 3524.3718 - val_accuracy: 0.0000e+00
Epoch 5/10
50/50 [==============================] - 852s 17s/step - loss: 3516.0393 - accuracy: 0.0000e+00 - val_loss: 3515.0708 - val_accuracy: 0.0000e+00
Epoch 6/10
50/50 [==============================] - 853s 17s/step - loss: 3515.0400 - accuracy: 0.0000e+00 - val_loss: 3515.0161 - val_accuracy: 0.0000e+00
Epoch 7/10
50/50 [==============================] - 853s 17s/ste

In [9]:
model.save('vgg-16-fine-tuned-last-2.h5')